In [2]:
import os

os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [19]:
import requests
from bs4 import BeautifulSoup

# 请求的 URL
url = 'https://timesofindia.indiatimes.com/topic/china-poverty-alleviation/0'

# 发送 GET 请求
response = requests.get(url)

# 使用 BeautifulSoup 解析 HTML 内容
soup = BeautifulSoup(response.content, 'html.parser')

# 找到所有的 <div class="uwU81">
divs = soup.find_all('div', class_='uwU81')

# 初始化列表以存储 href 和 span 文本
hrefs = []

# 遍历每个 div
for div in divs:
    # 在每个 div 下找到所有的 <span> 标签，并提取文本
    spans = div.find_all('span')
    for span in spans:
        span_text = span.get_text()
        # 检查 span 中是否包含 "中国"
        if 'China' in span_text or "Chinese" in span_text or "poverty" in span_text:
            # 在同一 div 下找到所有的 <a> 标签，并提取 href
            links = div.find_all('a', href=True)
            for link in links:
                href = link['href']
                if href not in hrefs:  # 检查链接是否已经存在
                    hrefs.append(href)

# 打印结果
print("HREFs:")
for href in hrefs:
    print(href)


HREFs:
https://timesofindia.indiatimes.com/india/indias-biggest-victory-on-poverty-in-5-charts/articleshow/94930625.cms
https://timesofindia.indiatimes.com/world/why-chinas-censors-are-deleting-videos-about-poverty/articleshow/100008678.cms
https://timesofindia.indiatimes.com/business/india-business/how-government-demonises-data-or-lack-of-it-for-political-ends/articleshow/100701732.cms
https://timesofindia.indiatimes.com/world/how-chinas-biggest-home-builder-became-its-greatest-risk/articleshow/103395438.cms
https://timesofindia.indiatimes.com/world/rest-of-world/poverty-climate-space-chinas-progress-in-10-years-under-xi-jinping/articleshow/94850978.cms
https://timesofindia.indiatimes.com/world/china/president-xi-claims-complete-victory-in-eradicating-absolute-poverty-in-china/articleshow/81210398.cms
https://timesofindia.indiatimes.com/world/china/india-china-key-to-global-poverty-alleviation/articleshow/65975015.cms
https://timesofindia.indiatimes.com/india/bipartisan-resolution-int

In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import os


# 检查并创建一个目录来保存文本文件
output_dir = "output_texts"
os.makedirs(output_dir, exist_ok=True)

# CSV 文件头
csv_headers = ['Title', 'Filename']

# 准备写入 CSV 文件
with open('titles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(csv_headers)

    # 遍历链接
    for index, url in enumerate(hrefs):
        try:
            # 发送 GET 请求
            response = requests.get(url)
            response.raise_for_status()  # 如果请求失败，将抛出异常

            # 使用 BeautifulSoup 解析 HTML 内容
            soup = BeautifulSoup(response.content, 'html.parser')
            #print(soup)
            
            # 查找指定的 div
            div = soup.find('div', class_ ='_s30J clearfix')
            if div:
                # 将 div 文本内容保存到文件
                with open(os.path.join(output_dir, f'{index}.txt'), 'w', encoding='utf-8') as file:
                    file.write(div.get_text())

                # 提取并保存标题
                title = soup.title.string if soup.title else 'No Title'
                csvwriter.writerow([f'{index}.txt',title ])
            else:
                print(f"在 URL {url} 中未找到指定的 div")

        except requests.RequestException:
            print(f"请求失败：{url}")


在 URL https://timesofindia.indiatimes.com/world/why-chinas-censors-are-deleting-videos-about-poverty/articleshow/100008678.cms 中未找到指定的 div
在 URL https://timesofindia.indiatimes.com/world/why-chinas-censors-are-deleting-videos-about-poverty/articleshow/100008678.cms 中未找到指定的 div
在 URL https://timesofindia.indiatimes.com/world/how-chinas-biggest-home-builder-became-its-greatest-risk/articleshow/103395438.cms 中未找到指定的 div
在 URL https://timesofindia.indiatimes.com/world/how-chinas-biggest-home-builder-became-its-greatest-risk/articleshow/103395438.cms 中未找到指定的 div
在 URL https://timesofindia.indiatimes.com/business/india-business/what-does-inclusion-being-key-to-growth-mean/articleshow/95834051.cms 中未找到指定的 div


In [28]:
import requests
from bs4 import BeautifulSoup
import os
import csv

# 初始化一个函数来处理每个页面
def process_page(page_url, csvwriter, start_index):
    try:
        # 发送 GET 请求
        response = requests.get(page_url)
        response.raise_for_status()  # 如果请求失败，将抛出异常

        # 使用 BeautifulSoup 解析 HTML 内容
        soup = BeautifulSoup(response.content, 'html.parser')

        # 找到所有的 <div class="uwU81">
        divs = soup.find_all('div', class_='uwU81')

        # 遍历每个 div
        for div in divs:
            try:
                spans = div.find_all('span')
                for span in spans:
                    span_text = span.get_text()
                    print(span_text)
                    if 'China' in span_text or 'Chinese' in span_text or 'poverty' in span_text:
                        print(0)
                        links = div.find_all('a', href=True)
                        for link in links:
                            href = link['href']
                            try:
                                # 处理每个链接
                                response = requests.get(href)
                                response.raise_for_status()  # 如果请求失败，将抛出异常

                                # 解析 HTML
                                soup = BeautifulSoup(response.content, 'html.parser')
                                div = soup.find('div', class_='_s30J clearfix')
                                if div:
                                    # 写入文件
                                    filename = os.path.join(output_dir, f'{start_index}.txt')
                                    with open(filename, 'w', encoding='utf-8') as file:
                                        file.write(div.get_text())

                                    # 写入 CSV
                                    title = soup.title.string if soup.title else 'No Title'
                                    csvwriter.writerow([title, filename])

                                    start_index += 1  # 增加索引以避免覆盖
                            except requests.RequestException:
                                print(f"请求失败：{href}")
            except AttributeError as e:
                print(f"解析错误：{e}")
    except requests.RequestException as e:
        print(f"页面请求失败：{page_url}, 错误: {e}")
    return start_index

# 检查并创建一个目录来保存文本文件
output_dir = "output_texts"
os.makedirs(output_dir, exist_ok=True)

# CSV 文件
with open('titles.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Title', 'Filename'])

    index = 0  # 初始索引
    # 遍历不同的页面
    for i in range(0, 7):  # 假设有 6 页
        page_url = f'https://timesofindia.indiatimes.com/topic/china-poverty-alleviation/{i}'
        print(page_url)
        index = process_page(page_url, csvwriter, index)


https://timesofindia.indiatimes.com/topic/china-poverty-alleviation/0
 India’s biggest victory on poverty in 5 charts 
0
 About 415mn people, the size of two Brazils, have come out of poverty in India over the past 15 years — the share of poor people down from over half to less than one in five. How has that changed lives? 
0
解析错误：'NoneType' object has no attribute 'find_all'
 How China defeats poverty — gag people, censor social media 
0
 Chinese President Xi Jinping says he has defeated poverty, but discussion of economic struggle is taboo, scrubbed from the internet and banished from the news. 
0
解析错误：'NoneType' object has no attribute 'find_all'
 How govt let 7.5 crore slip into poverty in one year 
0
 We need clean data before we can make any kind of decision, says economist, commentator and author Parakala Prabhakar. But what happens if the data is not credible? What happens when the government cherry-picks numbers to suit its political needs? 
 How China's biggest home builder b

 Why labour, not capital is key to India's 8% growth 
 Just like South Korea & China did, India must help its labour-intensive industries rise rapidly 
0
 China's Communist Party leadership reshuffle: What to look for 
0
 
 How China's Xi Jinping accumulated power, and why it matters in a third term 
0
 Xi, 69, is widely expected to break with precedent at the ruling Communist Party Congress that starts October 16 and extend his decade-long leadership for another five years - or beyond - cementing the party's resurgence across all aspects of China, with Xi officially its "core". 
0
 ‘Will always put Pakistan as a priority in our neighbourhood diplomacy': China to PM Sharif 
0
 
 COVID-19 to hit South Asia very hard, likely to wipe out gains made in poverty alleviation:World Bank 
0
 The global coronavirus pandemic will hit South Asia very hard and the significant gains made in poverty alleviation in the region are likely to be wiped out due to the impact of the deadly disease, the Worl

 China will resolutely defend sovereignty, push ahead with BRI in 2019: Xi 
0
 
 China building ‘road’ to colonization, warns Tibetan President in exile 
0
 
 Why high income tax rates are so unjust in India 
 As the govt points to other high-tax nations to justify levy on the super-rich, numbers show it hardly matches up to them on social welfare coverage 
 China's wealth gap: Teslas for the rich, footpaths for the poor 
0
 
 Pakistan, China refute report about rift over CPEC's 'unfairness' 
0
 Pakistan and China on Tuesday refuted a report in a UK daily that the former is rethinking its role in the China-Pakistan Economic Corridor (CPEC) as it was "unfair", reported newspaper Dawn. The article was carried by The Financial Times on Monday and was titled: "Pakistan rethinks its role in Xi's Belt and Road plan". 
0
 Hu raises poverty line by 92% before regime change 
0
 China has more than trebled the number of ‘poor' people by raising the poverty line by 92%. 
0
 Universal Basic Income

 NEW DELHI: Unimpressed by the steps taken in Pakistan to combat terrorism, India has termed them "inadequate" and dismissed Chinese calls for restraint, saying these should be directed to Pakistan. 
0
 Biden to focus on World Bank reform, new funding at G20 in New Delhi 
 US President Joe Biden will focus on reforming the World Bank and urging other multilateral development banks to boost lending for climate change and infrastructure projects during the G20 leaders summit in India, the White House said on Tuesday. 
 India's G20 positioning as voice for emerging economies a success: US official 
 Margaret MacLeod, Hindustani Spokesperson for the US Department of State, told PTI in an interview at the US Embassy in London ahead of her visit to New Delhi this week that the US is hopeful that sufficient common ground could be found at the conclusion of the G20 Summit.The US State Department also welcomed India stepping up with its chairman statements during the course of the G20 presidenc